# Generating tarot meanings, interpretations, and questions

Now that the GPT-2 model has been re-trained on the divination texts, it's time to start generating new tarot meanings, interpreations, and questions!



## Setup

In [ ]:
# GPU Check
!nvidia-smi

Fri Nov 27 23:53:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# %%capture
# !pip install transformers
import transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import gensim

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

Importing the saved GPT-2 model

In [ ]:
output_dir = '/content/drive/MyDrive/gpt2/models/model_2'

device = torch.device("cuda")

# to load model: 
gpt2model = GPT2LMHeadModel.from_pretrained(output_dir)
gpt2tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
gpt2model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

Some functions to generate some sample texts for preview and to save as a list for repeated use.

In [ ]:
def generate_samples(num_samples, prompt=None):

    gpt2model.eval()

    if prompt is None:
        prompt = "<|startoftext|>"

    generated = torch.tensor(gpt2tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)

    sample_outputs = gpt2model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=num_samples
                                )
    
    for i, sample_output in enumerate(sample_outputs):
        print("{}: {}\n".format(i, gpt2tokenizer.decode(sample_output, skip_special_tokens=True)))  

In [ ]:
generate_samples(10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: If you are the kind of Leo who prefers to wait for opportunities to shine you will be more inclined to see the good in others.

1: Don’t give up on it yet.

2: You want to prove your critics wrong and you won’t let anyone stop you.

3: But as the Sun moves into your opposite sign tomorrow you must also work more closely with those you have not been able to work with in recent weeks.

4: The universe is trying to trick you.

5: Whatever the reason, don’t waste time on it.

6: You won’t stop until everything is done.

7: If you make an effort to get out and about and meet up with other people this weekend you will accomplish something worthwhile.

8: Actions speak louder than words and if you are smart you will speak up over the next 24 hours.

9: For there is, as it were, no longer any need for the idea of oneself as a separate being, for this idea vanishes completely in its senseless complexity and unintelligibility.



In [ ]:
def generate_text_list(num_samples, prompt=None):

    text_list = []

    gpt2model.eval()

    if prompt is None:
        prompt = "<|startoftext|>"

    generated = torch.tensor(gpt2tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)

    sample_outputs = gpt2model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=num_samples
                                )
    
    for sample_output in sample_outputs:
        text_list.append(gpt2tokenizer.decode(sample_output, skip_special_tokens=True))

    return text_list    

## Generating texts

### Tarot meanings and interpretations

First, I'll generate meanings or interpretations for each card. I want to have some options for the 22 cards I'll be making, so hoping for roughly 10 meanings per card and then I can pick which ones I like best.

In [ ]:
meanings = generate_text_list(200)

for i, meaning in enumerate(meanings):
    print(f'{i}: {meaning}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: You may not think much of yourself but now is a good time to look more closely at what you are doing – maybe even thinking!
1: Try to keep your plans light over the next few days, especially if they are of a financial nature.
2: You don’t have to be in control all the time but if you are smart you will let the other person see that you still have limits, even if they are not quite as high as they would like them to be.
3: The only people who seem determined to get things done are those who know how to get on and get things done for themselves.
4: In fact there is a real possibility of a thousand years.
5: You won’t be able to hide your feelings today: your head will remain in your pocket.
6: Today’s lunar eclipse in your birth sign promises that you will have plenty of material and emotional junk to catch up on.
7: You have to make a decision, even if it is the wrong decision, and because others are reluctant to follow it you will have to take on too many tasks and too many people.


In [ ]:
idx_to_drop = [3, 4, 5, 6, 10, 11, 15, 16, 17, 20, 21, 22, 25, 26, 28, 29, 
               33, 34, 38, 40, 42, 45, 47, 49, 53, 54, 59, 60, 62, 66, 
               70, 71, 79, 80, 84, 86, 87, 88, 93, 95, 98, 101, 105, 106, 111, 112, 114, 119, 
               120, 121, 122, 129, 131, 132, 136, 141, 142, 149, 151, 152, 155, 156, 157, 
               161, 165, 169, 172, 176, 178, 184, 189, 190, 195, 196]

In [ ]:
meanings_df = pd.DataFrame(meanings, columns=['meanings'])
meanings_df.drop(index=idx_to_drop, inplace=True)
meanings_df

,meanings
0,You may not think much of yourself but now is ...
1,Try to keep your plans light over the next few...
2,You don’t have to be in control all the time b...
7,"You have to make a decision, even if it is the..."
8,And you'll be the life and soul of the party.
...,...
193,Others may complain that you are not doing eno...
194,The more you get done the more fun you have wi...
197,What makes you smile?
198,You may think it is okay to put your health fi...


126 docs is not enough, so I'll generate another batch...

In [ ]:
meanings2 = generate_text_list(200)

for i, meaning in enumerate(meanings2):
    print(f'{i}: {meaning}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Yes, you can do things that others don’t, but do you really need to rush them through like they are trying to slow down?
1: Some people may say that you are desperate but if you make the effort you will find it will pay off in ways you have not anticipated.
2: No matter how independent-minded you may be in the world you are advised not to take any drastic action today.
3: You have no limits, so long as you don’t take on too much.
4: Your mind will be open and honest today and that’s a huge plus.
5: A clash of egos could break out today and if you are wise you will sit back and watch the two feud for a few hours.
6: Some people seem to expect you to be the kind of person who always wants to be noticed but there is something you must learn about yourself.
7: Your way with words will make matters easier to understand.
8: This hexagram is a variation of the design of Mercury, and of the four trigrams in particular.
9: Don’t worry about whether someone is mad because there are enough fac

In [ ]:
idx_to_drop2 = [8, 16, 17, 18, 28, 29, 30, 31, 34, 36, 38, 39, 46, 
                 51, 53, 57, 62, 65, 67, 68, 69, 80, 83, 90, 94, 95, 97, 
                 103, 105, 106, 107, 108, 109, 110, 111, 112, 113, 
                 122, 124, 126, 128, 130, 132, 133, 136, 137, 141, 148, 149, 
                 161, 162, 163, 165, 167, 173, 176, 189, 194, 196, 198]

In [ ]:
meanings_df2 = pd.DataFrame(meanings2, columns=['meanings'])
meanings_df2.drop(index=idx_to_drop2, inplace=True)
meanings_df2

,meanings
0,"Yes, you can do things that others don’t, but ..."
1,Some people may say that you are desperate but...
2,No matter how independent-minded you may be in...
3,"You have no limits, so long as you don’t take ..."
4,Your mind will be open and honest today and th...
...,...
192,You know what needs to be done.
193,You seem to be rushing away from what you are ...
195,Don’t let your fears get the better of you ove...
197,Try not to be so open and honest with other pe...


In [ ]:
meanings_list = list(set(meanings_df.meanings.tolist() + meanings_df2.meanings.tolist()))
print(f'Total meanings: {len(meanings_list)}')
print(f'Number of meanings per card: {len(meanings_list)/22:.2f}')

Total meanings: 266
Number of meanings per card: 12.09


This should be sufficient - now to save these as a txt file for later use.

In [ ]:
# save for later use
with open("/content/drive/MyDrive/gpt2/tarot_meanings.txt", mode='w', encoding='utf-8') as myfile:
    myfile.write('\n'.join(meanings_list))

### Questions to ask

Tarot cards typically raise additional questions to ponder as part of the process. 

I'll seed the model with the following question prompt words below:

In [ ]:
question_prefixes = ['Who will', 'Who is', 'Who can', 'Who has',
                     'What is', 'What if', 'What would', 'What do', 'What assumptions', 
                     'When will', 'When can',
                     'Where can', 'Where is', 'Where should',
                     'Why is', 'Why are', 'Why did', 'Why should',
                     'How can I', 'How might', 'How is', 'How do', 
                     'Should they', 'Should I', 
                     'Would I', 'Would they',
                     'Can I', 'Can they',
                     'Is it', 'Is there'
                    ]

len(question_prefixes)               

30

In [ ]:
questions = []
num_q = 10


for prompt in question_prefixes:
    q_list = generate_text_list(num_q, prompt=prompt)
    questions.append(q_list)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
questions[0]

['Who will take the world by storm over the next 48 hours.',
 'Who will be back in full force again soon.',
 'Who will they be trying to trick you?',
 'Who will you play the role of wise guy or woman?',
 'Who will you give me more freedom and responsibilities later on?',
 'Who will go out of your way to make it seem as if everyone is as stupid as you!',
 'Who will you get the breakthrough you crave?',
 'Who will make you smile.',
 'Who will be fun.',
 'Who will be watching over you very soon.']

In [ ]:
questions = list(set([q for sublist in questions for q in sublist]))
len(questions)

297

In [ ]:
for i, q in enumerate(questions):
    print(f'{i}: {q}')

0: Would they feel for having such a big idea.
1: Who is it worth fighting for?
2: What if you are the kind of Cancerian who likes to stay within logical bounds the planets indicate you will be rewarded in some way for your efforts.
3: Who has not been found
4: Should they are not really that bad.
5: Why are you really that good at being tough?
6: Where is not what thou sawest, and that thou shouldest fear no more.
7: What would you do if you could?
8: What assumptions govern my daily activities and I don’t understand why other people choose to ignore me when so many have the opportunity to give me something or someone.
9: When will you go it alone?
10: How do you try something different or what is it that takes you?
11: Where is it permissible to eat money after all?
12: Where should you listen to them?
13: Can I’ve had two cups of coffee in my hand for several weeks.
14: How do you care about it?
15: Would I have never had a bird in my mouth.
16: When can you take responsibility for 

In [ ]:
q_df = pd.DataFrame(questions, columns=['questions'])
q_df.head()

,questions
0,Would they feel for having such a big idea.
1,Who is it worth fighting for?
2,What if you are the kind of Cancerian who like...
3,Who has not been found
4,Should they are not really that bad.


After reiewing *every* question generated, I'll drop the following:

In [ ]:
q_to_drop = [0, 2, 4, 6, 8, 10, 11, 12, 13, 15, 17, 18, 
             20, 21, 22, 26, 28, 29, 30, 31, 32, 35, 39, 
             41, 43, 46, 48, 49, 51, 53, 56, 59, 60, 63, 64, 65, 
             70, 71, 76, 77, 78, 79, 84, 85, 87, 88, 89, 93, 95, 96, 98, 
             104, 105, 108, 109, 111, 113, 114, 115, 116, 118, 
             121, 121, 122, 123, 124, 125, 129, 130, 132, 135, 137, 138, 
             148, 152, 153, 155, 158, 159, 160, 161, 165, 166, 169, 
             171, 172, 173, 174, 180, 184, 187, 188, 189, 190, 192, 193, 196, 
             201, 202, 204, 205, 208, 210, 212, 213, 214, 215, 216, 217, 218, 219, 
             221, 225, 226, 227, 230, 232, 233, 234, 236, 237, 
             242, 243, 244, 245, 246, 247, 248, 250, 251, 252, 254, 255, 256, 258, 
             260, 265, 267, 269, 271, 273, 275, 276, 280, 281, 282, 284, 287, 289]

In [ ]:
q_df = q_df.drop(index=q_to_drop)

In [ ]:
q_df

,questions
1,Who is it worth fighting for?
3,Who has not been found
5,Why are you really that good at being tough?
7,What would you do if you could?
9,When will you go it alone?
...,...
292,Who will make you smile.
293,What assumptions govern our lives and our way ...
294,What is the best revenge for your feelings of ...
295,Why did I say something just to make a point?


Some of these don't have question marks at the end, so I'll replace any ending punctuation with a '?'

In [ ]:
def make_question(text):
    if text[-1] != '?':
        text = text[:-1]
        text += '?'       
    return text    

In [ ]:
q_df['questions'] = q_df['questions'].map(make_question)

In [ ]:
q_df

,questions
1,Who is it worth fighting for?
3,Who has not been foun?
5,Why are you really that good at being tough?
7,What would you do if you could?
9,When will you go it alone?
...,...
292,Who will make you smile?
293,What assumptions govern our lives and our way ...
294,What is the best revenge for your feelings of ...
295,Why did I say something just to make a point?


In [ ]:
print(f'Questions per card: {len(q_df)/22:.2f}')

Questions per card: 6.77


I'll want at most 3 questions per card, so this should be sufficient.

In [ ]:
q = q_df.questions.tolist()

with open("/content/drive/MyDrive/gpt2/questions.txt", mode='w', encoding='utf-8') as myfile:
    myfile.write('\n'.join(q))